# Experiments

Just to be clear, the following examples are for demonstration and testing purposes only. Users are not supposed to make their own dummy batches using their own data. Refer to the sections *[High-Level API](api_high_level.ipynb)* and/ or *[Low-Level API](api_low_level.ipnyb)* if you actually want to use the library. 

## Prerequisites
* If you've already completed the *[Installation](installation.ipynb)* section, then let's get started.
* This section makes use of the data in the *[Example Datasets](example_datasets.ipynb)* section.

In [4]:
import aiqc
from aiqc import examples

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Dummy Experiments

The `list_demo_batches()` method describes the predefined experiments that are available.

> By default it returns a Pandas DataFrame, but you can `list_demo_files(format='list')` to change that.

In [8]:
examples.list_demo_batches()

,batch_name,supervision,analysis,sub_analysis,validation,dataset
0,multiclass,supervised,classification,multi label,validation split,iris.tsv
1,binary,supervised,classification,binary,validation split,sonar.csv
2,regression,supervised,regression,None,validation split,houses.csv
3,multiclass_folded,supervised,classification,multi label,cross-folds,iris_10x.tsv


* Each `Batch` contains predefined training `Jobs` and a `Hyperparamset`.
  * Each `Job` is comprised of:
    * An `Algorithm` and a unique `Hyperparamcombo`.
    * They gets fit to a `Datasets` made from a dummy data file seen above.
      * More specifically, its `Labels, Featuresets` are sliced into `Splitsets, Foldsets`.

## Example: supervised regression

The entries from the `name` column above, can be fed into the `make_demo_batch()` method.

In [9]:
batch = examples.make_demo_batch('regression')

### ORM in action

It's easy to dig into the Batches using the ORM to see how they are constructed.

In [10]:
batch.job_count

4

In [11]:
batch.jobs[3].hyperparamcombo.hyperparameters

{'neuron_count': 48, 'epochs': 75}

In [12]:
batch.splitset.featureset.dataset.shape

{'rows': 506, 'columns': 13}

In [13]:
batch.algorithm.function_model_train

<function aiqc.examples.regression_model_train(model, samples_train, samples_evaluate, **hyperparameters)>

### Hyperparameter tuning in action

When you execute a Batch of Jobs, it will run the jobs on a background process, so you can continue to work in your Python shell.

> It does this by spawning a `multiprocessing.Process` that lives until each Job is completed or the process is killed.

In [14]:
batch.run_jobs()

🔮 Training Models 🔮:  50%|█████████████████████                     | 2/4 [00:13<00:12,  6.29s/it]

The execution of the Jobs can easily be stopped, and then resumed at the Job that was previously running.

In [15]:
batch.stop_jobs()


Killed `multiprocessing.Process` 'aiqc_batch_9' spawned from Batch <id:9>



In [16]:
batch.run_jobs()


Resuming jobs...



🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 4/4 [00:13<00:00,  3.37s/it]


### Inspecting the Result of an Individual Job

When each Job completes, it will create a `Result` that contains `predictions` and `metrics` attributes for each split/ fold.

In [17]:
batch.jobs[3].results[0].metrics

{'test': {'r2': 0.6963792716428294,
  'mse': 0.2951804864898259,
  'explained_variance': 0.7063090130204511,
  'loss': 0.29518043994903564},
 'validation': {'r2': 0.7381085765314983,
  'mse': 0.2410270741642624,
  'explained_variance': 0.7466099877263364,
  'loss': 0.2410270869731903},
 'train': {'r2': 0.734697880657778,
  'mse': 0.26530211934222203,
  'explained_variance': 0.7404845093871112,
  'loss': 0.2653021216392517}}

The JSON values of these attributes are interoperable with Python dictionaries.

In [18]:
batch.jobs[3].results[0].predictions['validation']

array([[-0.15168118],
       [ 1.6077876 ],
       [ 0.3561042 ],
       [ 0.08792061],
       [ 2.1205187 ],
       [-0.22818494],
       [-0.27557564],
       [-0.44030523],
       [-0.07237878],
       [ 0.310706  ],
       [ 0.28391394],
       [ 0.0256266 ],
       [ 0.9612344 ],
       [ 1.6431339 ],
       [-0.4306181 ],
       [-0.31357193],
       [ 0.3742989 ],
       [ 0.19994563],
       [-0.622221  ],
       [-0.6667788 ],
       [-0.43117243],
       [ 0.25616717],
       [-0.11828876],
       [-0.24248827],
       [-0.43472743],
       [ 0.4519846 ],
       [-0.3030482 ],
       [ 0.78143   ],
       [ 0.07700422],
       [-0.11437604],
       [-0.68678147],
       [ 0.10900812],
       [-0.22920918],
       [-0.42426032],
       [-0.39213717],
       [ 2.0510547 ],
       [ 0.8801753 ],
       [ 0.8849825 ],
       [ 0.26650494],
       [-0.10267839],
       [ 0.07201514],
       [-0.20041507],
       [ 0.16122897],
       [ 0.16937049],
       [ 0.05018368],
       [-0

### Inspecting the metrics of the entire Batch

In [20]:
batch.metrics_to_pandas()

,job_id,split,r2,mse,explained_variance,loss
0,33,test,0.664181,0.326484,0.766747,0.326484
1,33,validation,0.694890,0.280802,0.801747,0.280802
2,33,train,0.721822,0.278178,0.793746,0.278178
3,34,test,0.703963,0.287808,0.709323,0.287808
4,34,validation,0.719353,0.258289,0.729316,0.258289
5,34,train,0.722841,0.277159,0.738167,0.277159
6,35,test,0.734238,0.258374,0.734808,0.258374
7,35,validation,0.743797,0.235792,0.748372,0.235792
8,35,train,0.779638,0.220362,0.785239,0.220362
9,36,test,0.696379,0.295180,0.706309,0.295180


## Example: supervised multi-classification with cross-fold validation

The following method uses *demo_file* 'iris_10x.tsv', so if you see duplicate samples that is why.

In [22]:
batch = examples.make_demo_batch('multiclass_folded')

In [23]:
batch.run_jobs()

🔮 Training Models 🔮: 100%|████████████████████████████████████████| 10/10 [00:51<00:00,  5.18s/it]


In [24]:
batch.job_count

10

In [25]:
batch.metrics_to_pandas()

,job_id,split,roc_auc,accuracy,precision,recall,f1,loss
0,37,test,0.999822,0.995556,0.995614,0.995556,0.995555,0.027901
1,37,folds_train_combined,0.999638,0.994048,0.994151,0.994048,0.994047,0.034026
2,37,fold_validation,0.999460,0.990476,0.990752,0.990476,0.990478,0.037803
3,38,test,1.000000,0.997778,0.997792,0.997778,0.997778,0.015914
4,38,folds_train_combined,1.000000,0.992857,0.993008,0.992857,0.992857,0.021590
5,38,fold_validation,1.000000,0.995238,0.995304,0.995238,0.995237,0.023279
6,39,test,0.999793,0.940000,0.949153,0.940000,0.939510,0.089607
7,39,folds_train_combined,0.999669,0.938095,0.947880,0.938095,0.937595,0.100268
8,39,fold_validation,0.998013,0.947619,0.954478,0.947619,0.947186,0.123960
9,40,test,0.999837,0.957778,0.962525,0.957778,0.957608,0.063986


### API Details

For more detailed information reference the *[Low-Level API](api_low_level.ipynb)* documentation.

### Metrics & Visualization

For more information of visualization of performance metrics, reference the *[Visualization & Metrics](api_low_level.ipynb)* documentation.